In [93]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score, roc_auc_score

In [83]:
!pip install category_encoders

     ---------------------------------------- 81.9/81.9 kB 1.6 MB/s eta 0:00:00


In [2]:
os.chdir("C:\\Users\\HP\\Desktop\\code challenges\\datasets")

In [72]:
df = pd.read_csv("titanic.csv")

In [73]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [75]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)

In [77]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [79]:
X = df.drop(columns='Survived')
y = df['Survived']


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [81]:
# target encoding

In [85]:
target_encoder = TargetEncoder(cols=['Sex', 'Embarked'])

In [86]:
X_train_encoded = target_encoder.fit_transform(X_train, y_train)
X_test_encoded = target_encoder.transform(X_test)

In [87]:
X_train_encoded.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
692,3,0.185185,28.0,0,0,56.4958,0.330116
481,2,0.185185,28.0,0,0,0.0000,0.330116
527,1,0.185185,28.0,0,0,221.7792,0.330116
855,3,0.743083,18.0,0,1,9.3500,0.330116
801,2,0.743083,31.0,1,1,26.2500,0.330116


In [88]:
# frequency encoding

In [89]:
def frequency_encoding(column):
    freq_encoding = column.value_counts() / len(column)
    return column.map(freq_encoding)

In [90]:
X_train_encoded['Pclass_freq'] = frequency_encoding(X_train['Pclass'])
X_test_encoded['Pclass_freq'] = frequency_encoding(X_test['Pclass'])

In [91]:
X_train_encoded.drop(columns='Pclass', inplace=True)
X_test_encoded.drop(columns='Pclass', inplace=True)

In [92]:
X_train_encoded.head()

,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_freq
692,0.185185,28.0,0,0,56.4958,0.330116,0.549157
481,0.185185,28.0,0,0,0.0000,0.330116,0.210674
527,0.185185,28.0,0,0,221.7792,0.330116,0.240169
855,0.743083,18.0,0,1,9.3500,0.330116,0.549157
801,0.743083,31.0,1,1,26.2500,0.330116,0.210674


In [61]:
# model training and evaluation

In [94]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [95]:
model.fit(X_train_encoded, y_train)


RandomForestClassifier(random_state=42)

In [96]:
y_pred = model.predict(X_test_encoded)
y_prob = model.predict_proba(X_test_encoded)[:, 1]

In [97]:
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

In [98]:
print(f'Accuracy: {accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

Accuracy: 0.8101
ROC AUC: 0.8370
